## Preprocessing and Pipelines

We will lean how to preprocess data and create machine learning pipelines using scikit-learn.

In [ ]:
# Import core data manipulation libraries
import numpy as np
import pandas as pd

# Import scikit-learn utilities for model training and evaluation
from sklearn.model_selection import train_test_split  # Splits data into train/test sets
from sklearn.linear_model import LogisticRegression  # Binary classification model
from sklearn.metrics import accuracy_score  # Measures prediction accuracy

## Importing the Titanic Dataset

In this section we will import the Titanic dataset to demonstrate preprocessing and pipeline creation.

In [ ]:
# Load the Titanic dataset from seaborn's built-in datasets
# This dataset contains passenger info and whether they survived
from seaborn import load_dataset

titanic = load_dataset("titanic")
titanic.head()  # Preview the first 5 rows

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
# Get summary statistics for numeric columns (count, mean, std, min, max, quartiles)
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
# Display column names, data types, and non-null counts
# Useful for identifying missing values and data types
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [ ]:
# Remove the "deck" feature - it has too many missing values (77% null)
# Dropping columns with excessive missing data avoids imputation bias
titanic.drop(columns="deck", inplace=True)
titanic.head()

In [ ]:
# Handle missing age values by filling with the mean (imputation)
# Mean imputation preserves the overall distribution while filling gaps
mean_age = titanic["age"].mean()
titanic["age"] = titanic["age"].fillna(value=mean_age)
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB


In [ ]:
# Drop rows with remaining missing values (only 2 rows missing "embark_town")
# When very few rows have missing data, dropping is simpler than imputation
titanic.dropna(inplace=True)
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     889 non-null    int64   
 1   pclass       889 non-null    int64   
 2   sex          889 non-null    object  
 3   age          889 non-null    float64 
 4   sibsp        889 non-null    int64   
 5   parch        889 non-null    int64   
 6   fare         889 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        889 non-null    category
 9   who          889 non-null    object  
 10  adult_male   889 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        889 non-null    object  
 13  alone        889 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 86.1+ KB


In [ ]:
# Remove redundant columns that duplicate information already captured elsewhere
# "embarked" duplicates "embark_town", "class" duplicates "pclass", etc.
# Reducing redundancy simplifies the model and prevents multicollinearity
titanic.drop(columns=["embarked", "class", "who", "adult_male", "alive"], inplace=True)
titanic.info()

In [ ]:
# Convert categorical "sex" to numeric using label encoding
# ML algorithms require numeric input; 0 = female, 1 = male
titanic["is_male"] = titanic["sex"].map({"female": 0, "male": 1})
titanic.head()

C:\Users\Owner\AppData\Local\Temp\ipykernel_39080\3052910026.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  titanic["is_male"] = titanic["sex"].replace({"female": 0, "male": 1})


,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone,is_male
0,0,3,male,22.0,1,0,7.2500,Southampton,False,1
1,1,1,female,38.0,1,0,71.2833,Cherbourg,False,0
2,1,3,female,26.0,0,0,7.9250,Southampton,True,0
3,1,1,female,35.0,1,0,53.1000,Southampton,False,0
4,0,3,male,35.0,0,0,8.0500,Southampton,True,1


In [ ]:
# Remove the original "sex" column since we now have the numeric "is_male"
titanic.drop(columns=["sex"], inplace=True)
titanic.head()

,survived,pclass,age,sibsp,parch,fare,embark_town,alone,is_male
0,0,3,22.0,1,0,7.2500,Southampton,False,1
1,1,1,38.0,1,0,71.2833,Cherbourg,False,0
2,1,3,26.0,0,0,7.9250,Southampton,True,0
3,1,1,35.0,1,0,53.1000,Southampton,False,0
4,0,3,35.0,0,0,8.0500,Southampton,True,1


In [ ]:
# Calculate correlation matrix to identify relationships between features
# Values close to +1 or -1 indicate strong relationships; close to 0 means weak
titanic.corr(numeric_only=True)

,survived,pclass,age,sibsp,parch,fare,alone,is_male
survived,1.000000,-0.335549,-0.074673,-0.034040,0.083151,0.255290,-0.206207,-0.541585
pclass,-0.335549,1.000000,-0.327954,0.081656,0.016824,-0.548193,0.138553,0.127741
age,-0.074673,-0.327954,1.000000,-0.231875,-0.178232,0.088604,0.177712,0.089434
sibsp,-0.034040,0.081656,-0.231875,1.000000,0.414542,0.160887,-0.584186,-0.116348
parch,0.083151,0.016824,-0.178232,0.414542,1.000000,0.217532,-0.583112,-0.247508
fare,0.255290,-0.548193,0.088604,0.160887,0.217532,1.000000,-0.274079,-0.179958
alone,-0.206207,0.138553,0.177712,-0.584186,-0.583112,-0.274079,1.000000,0.306985
is_male,-0.541585,0.127741,0.089434,-0.116348,-0.247508,-0.179958,0.306985,1.000000


#### What "low correlation" means:
- Values close to 0 (like -0.07, -0.04, 0.08) indicate almost no linear relationship
- These features don't help predict survival much on their own
- age, sibsp, and parch all have correlations under ±0.1

#### What the sign tells you:
- Positive (+0.26): As fare goes up, survival goes up
- Negative (-0.54): As is_male goes up (male), survival goes down

#### Why drop them?
- They add complexity without adding predictive power
- Simpler models often generalize better
- Fewer features = faster training, less overfitting

Caveat: Low correlation doesn't always mean useless. Age might matter in combination with other features (e.g., young children with parents). But for a simple logistic regression, dropping low-correlation features is a reasonable choice.

#### What matters for dropping features:
- The absolute value (distance from 0)
- |-0.54| = 0.54 → strong relationship
- |-0.07| = 0.07 → weak relationship

In [ ]:
# Drop features with low correlation to "survived" (absolute value < 0.1)
# Also drop "embark_town" since it's categorical and not yet encoded
# Keeping only relevant features reduces noise and improves model performance
titanic.drop(columns=["age", "sibsp", "parch", "embark_town"], inplace=True)

In [ ]:
# Verify the cleaned dataset - should have: survived, pclass, fare, is_male
titanic.head()

,survived,pclass,fare,alone,is_male
0,0,3,7.2500,False,1
1,1,1,71.2833,False,0
2,1,3,7.9250,True,0
3,1,1,53.1000,False,0
4,0,3,8.0500,True,1


### Standardizing the Values
It is useful to standardize features so they have a mean of 0 and a standard deviation of 1. This ensures that all features contribute equally to the distance calculations in algorithms like logistic regression.

In [ ]:
# Split data into features (X) and target (y)
# X = what we use to predict, y = what we're predicting (survived)
y = titanic["survived"]
X = titanic.drop(columns=["survived"])

# Split into training (67%) and testing (33%) sets
# random_state ensures reproducible results
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [ ]:
# Import StandardScaler to normalize features to mean=0, std=1
# This prevents features with large values from dominating the model
from sklearn.preprocessing import StandardScaler

In [ ]:
# Create and fit the scaler on training data only
# fit() learns the mean and variance from training data
scaler = StandardScaler()
scaler.fit(X_train)

,copy,True
,with_mean,True
,with_std,True


In [ ]:
# Inspect the learned statistics from the training data
# These values will be used to transform both train and test sets
print("Mean per feature:", scaler.mean_)
print("Variance per feature:", scaler.var_)

array([6.87550314e-01, 2.64921948e+03, 2.41299343e-01, 2.24238401e-01])

In [ ]:
# Apply the same scaling transformation to both train and test sets
# Important: Use transform() only (not fit_transform) on test data
# This ensures test data is scaled using training statistics
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Verify the scaled output - values should now be centered around 0
# Notice the output is a numpy array, not a DataFrame
X_train_scaled

array([[-1.60124536,  0.37097199,  0.82798092,  0.71693438],
       [-0.39524411, -0.38407115,  0.82798092, -1.39482779],
       [-1.60124536,  0.95374775, -1.2077573 ,  0.71693438],
       ...,
       [-1.60124536, -0.13287517,  0.82798092, -1.39482779],
       [ 0.81075714,  0.03121472, -1.2077573 , -1.39482779],
       [ 0.81075714, -0.46850387,  0.82798092,  0.71693438]],
      shape=(595, 4))

The flow:
- `fit(X_train)` - Learn the statistics (mean, variance) from the training data
- `transform(X_train)` - Apply those statistics to scale the data

Think of it like this:
- fit()       →  "Measure the ruler" (learn mean=30, std=10)
- transform() →  "Use the ruler to measure" (apply the scaling formula)


In [ ]:
# Create and train the logistic regression model
# fit() learns the relationship between scaled features and survival
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)  # X is scaled, y stays as 0/1

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [ ]:
# Evaluate the model on unseen test data
# predict() returns 0 or 1 for each test sample
y_pred = log_reg.predict(X_test_scaled)

# Calculate accuracy: (correct predictions) / (total predictions)
accuracy_score(y_test, y_pred)

0.7993197278911565

In [ ]:
# Calculate the baseline accuracy (survival rate in the dataset)
# If we predicted "died" for everyone, we'd be right ~62% of the time
# Our model should beat this baseline to be considered useful
titanic["survived"].mean()  # ~0.38 survived, so predicting 0 = 62% accuracy

np.float64(0.38245219347581555)

## Create a Pipeline for Preprocessing and Modeling
We will create a pipeline that combines preprocessing steps (like standardization) with a machine learning model (like logistic regression). This allows us to streamline the workflow and ensure that all steps are applied consistently during training and evaluation.

In [ ]:
# Import the Pipeline object
from sklearn.pipeline import Pipeline

In [ ]:
# Create a pipeline
pipeline = Pipeline(
    [("scaler", StandardScaler()), ("log_reg", LogisticRegression())], verbose=True
)

In [43]:
# Fit the pipeline
pipeline.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ........... (step 2 of 2) Processing log_reg, total=   0.0s


,steps,"[('scaler', ...), ('log_reg', ...)]"
,transform_input,None
,memory,None
,verbose,True
,copy,True
,with_mean,True
,with_std,True
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0


In [ ]:
# View all the steps
pipeline.named_steps

{'scaler': StandardScaler(), 'log_reg': LogisticRegression()}

In [47]:
# View mean
pipeline.named_steps["scaler"].mean_

array([ 2.32773109, 32.76836067,  0.59327731,  0.6605042 ])

In [48]:
# Predict with auto scaling of X_test dataset
pipeline.predict(X_test)

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0])

In [49]:
pipeline.score(X_test, y_test)

0.7993197278911565